In [1]:
import openai
openai.api_key = ''

In [2]:
import pickle

import pandas as pd

In [3]:
df = pd.read_csv('sigcse_2024.csv')

train_df = df[df.subset == 'train'].copy()
test_df = df[df.subset == 'test'].copy()

In [4]:
prompt_base = '''Pretend that you are a teacher and you are teaching students Python. You ask students to describe in plain English what the following piece of code does with the assumption that <assumption>:

<code>

Example correct answers:
<correct>

Example incorrect answers:
<incorrect>

One student answered:
<response>

Is the student’s answer complete, correct, unambiguous, and summarizing without details of implementations? (only *Yes.*/*No.*)
'''

In [5]:
gpt_responses = []
count = 0
for _, row in test_df.iterrows():
    print(f'{count}/{len(test_df)}')
    count += 1
    
    code = row['code']
    assumption = row['assumption'][12:-1] if '.' in row['assumption'] else row['assumption'][12:]
    response = row['response']
    
    if len(train_df[(train_df.qid == row['qid']) & (train_df.binary_ground_truth == 1)]) >= 5:
        correct = '\n'.join(map(str, train_df[(train_df.qid == row['qid']) & (train_df.binary_ground_truth == 1)].sample(5, random_state=32)['response']))
    elif len(train_df[(train_df.qid == row['qid']) & (train_df.binary_ground_truth == 1)]) >= len(row['example_correct_answers']):
        correct = '\n'.join(map(str, train_df[(train_df.qid == row['qid']) & (train_df.binary_ground_truth == 1)]['response']))
    if len(train_df[(train_df.qid == row['qid']) & (train_df.binary_ground_truth == 0)]) >= 5:
        incorrect = '\n'.join(map(str, train_df[(train_df.qid == row['qid']) & (train_df.binary_ground_truth == 0)].sample(5, random_state=32)['response']))
    else:
        incorrect = '\n'.join(map(str, train_df[(train_df.qid == row['qid']) & (train_df.binary_ground_truth == 0)]['response']))
    
    prompt = prompt_base.replace('<code>', code).replace('<assumption>', assumption).replace('<correct>', correct).replace('<incorrect>', incorrect).replace('<response>', response)
    if assumption == '':
        prompt.replace(' with the assumption that ', '')
    
    done = False
    while not done:
        try:
            gpt_response = openai.ChatCompletion.create(
                model='gpt-4-0314',
                messages=[{'role': 'system', 'content': prompt}],
                temperature=0
            )
        except Exception:
            continue

        gpt_responses.append(gpt_response['choices'][0]['message']['content'])
        
        done = True

0/620
1/620
2/620
3/620
4/620
5/620
6/620
7/620
8/620
9/620
10/620
11/620
12/620
13/620
14/620
15/620
16/620
17/620
18/620
19/620
20/620
21/620
22/620
23/620
24/620
25/620
26/620
27/620
28/620
29/620
30/620
31/620
32/620
33/620
34/620
35/620
36/620
37/620
38/620
39/620
40/620
41/620
42/620
43/620
44/620
45/620
46/620
47/620
48/620
49/620
50/620
51/620
52/620
53/620
54/620
55/620
56/620
57/620
58/620
59/620
60/620
61/620
62/620
63/620
64/620
65/620
66/620
67/620
68/620
69/620
70/620
71/620
72/620
73/620
74/620
75/620
76/620
77/620
78/620
79/620
80/620
81/620
82/620
83/620
84/620
85/620
86/620
87/620
88/620
89/620
90/620
91/620
92/620
93/620
94/620
95/620
96/620
97/620
98/620
99/620
100/620
101/620
102/620
103/620
104/620
105/620
106/620
107/620
108/620
109/620
110/620
111/620
112/620
113/620
114/620
115/620
116/620
117/620
118/620
119/620
120/620
121/620
122/620
123/620
124/620
125/620
126/620
127/620
128/620
129/620
130/620
131/620
132/620
133/620
134/620
135/620
136/620
137/620
138/62

In [6]:
predicted = list(map(lambda x : int('Yes' in x) if 'Yes' in x or 'No' in x else None, gpt_responses))
assert None not in predicted

In [7]:
pickle.dump(predicted, open('gpt4_0314_random_5p5n_examples.pkl', 'wb'))